# interfaces

> This module defines the API and the GUI of *findmycells*

In [ ]:
#| default_exp interfaces

In [ ]:
#| export

from abc import ABC, abstractmethod
from pathlib import Path, PosixPath
from typing import List, Dict, Tuple, Optional, Union
from traitlets.traitlets import MetaHasTraits as WidgetType

from tqdm.notebook import tqdm
from datetime import datetime
import pickle
import ipywidgets as w
from IPython.display import display
from ipyfilechooser import FileChooser

from findmycells.configs import ProjectConfigs
from findmycells.database import Database
from findmycells.core import ProcessingStrategy, ProcessingObject
from findmycells.preprocessing.specs import PreprocessingStrategy, PreprocessingObject

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class API:
    
    def __init__(self, project_root_dir: PosixPath) -> None:
        assert type(project_root_dir) == PosixPath, '"project_root_dir" must be pathlib.Path object referring to an existing directory.'
        assert project_root_dir.is_dir(), '"project_root_dir" must be pathlib.Path object referring to an existing directory.'
        self.project_configs = ProjectConfigs(root_dir = project_root_dir)
        self.database = Database(project_configs = self.project_configs)
        
        
    def update_database_with_current_source_files(self) -> None:
        self.database.compute_file_infos()
        
        
    def preprocess(self,
                   strategies: List[PreprocessingStrategy],
                   strategy_configs: Optional[List[Dict]]=None,
                   processing_configs: Optional[Dict]=None,
                   microscopy_reader_configs: Optional[Dict]=None,
                   roi_reader_configs: Optional[Dict]=None,
                   file_ids: Optional[List[str]]=None
                  ) -> None:
        processing_step_id = 'preprocessing'
        strategy_configs, processing_configs, file_ids = self._assert_and_update_input(processing_step_id = processing_step_id,
                                                                                       strategies = strategies,
                                                                                       strategy_configs = strategy_configs,
                                                                                       processing_configs = processing_configs,
                                                                                       file_ids = file_ids)
        microscopy_reader_configs = self._assert_and_update_reader_configs_input(reader_type = 'microscopy_images', reader_configs = microscopy_reader_configs)
        roi_reader_configs = self._assert_and_update_reader_configs_input(reader_type = 'rois', reader_configs = roi_reader_configs)
        for file_id in tqdm(file_ids, display = processing_configs['show_progress']):
            preprocessing_object = PreprocessingObject()
            preprocessing_object.prepare_for_processing(file_ids = [file_id], database = self.database)
            preprocessing_object.load_image_and_rois(microscopy_reader_configs = microscopy_reader_configs, roi_reader_configs = roi_reader_configs)
            preprocessing_object.run_all_strategies(strategies = strategies, strategy_configs = strategy_configs)
            preprocessing_object.save_preprocessed_images_on_disk()
            preprocessing_object.save_preprocessed_rois_in_database()
            preprocessing_object.update_database()
            del preprocessing_object
            if processing_configs['autosave'] == True:
                self.save_status()
                self.load_status()
                
                
    def _assert_and_update_reader_configs_input(self, reader_type: str, reader_configs: Optional[Dict]) -> Dict:            
        if reader_configs == None:
            if hasattr(self.project_configs, reader_type) == False:
                self.project_configs.add_reader_configs(reader_type = reader_type)
            reader_configs = getattr(self.project_configs, reader_type)
        else:
            assert type(reader_configs) == dict, f'"reader_configs" (data type: {reader_type}) has to be a dictionary!'
            default_configs = self.project_configs.default_configs_of_available_data_readers[reader_type]
            default_configs.assert_user_input(user_input = reader_configs)
            reader_configs = default_configs.fill_user_input_with_defaults_where_needed(user_input = reader_configs)
            self.project_configs.add_reader_configs(reader_type = reader_type, reader_configs = reader_configs)
        return reader_configs
    

    def segment(self,
                   strategies: List[PreprocessingStrategy],
                   strategy_configs: Optional[List[Dict]]=None,
                   processing_configs: Optional[Dict]=None,
                   file_ids: Optional[List[str]]=None
                  ) -> None:
        processing_step_id = 'segmentation'
        strategy_configs, processing_configs, file_ids = self._assert_and_update_input(processing_step_id = processing_step_id,
                                                                                       strategies = strategies,
                                                                                       strategy_configs = strategy_configs,
                                                                                       processing_configs = processing_configs,
                                                                                       file_ids = file_ids)
        # ToDo: split file_ids in batches if requested in processing_configs
        
        for file_id in tqdm(file_ids, display = processing_configs['show_progress']):
            segmentation_object = SegmentationObject()
            # Continue here
            # information on previous implementation can be found in:
            #   /tmp_nbs/03_api.ipynb - class: "Project", method "segment"
            
            segmentation_object.prepare_for_processing(file_ids = [file_id], database = self.database)
            segmentation_object.run_all_strategies(strategies = strategies, strategy_configs = strategy_configs)
            segmentation_object.update_database()
            del segmentation_object
            if processing_configs['autosave'] == True:
                self.save_status()
                self.load_status()
                
                

    
    def save_status(self) -> None:
        date = f'{datetime.now():%Y_%m_%d}'
        dbase_filename = f'{date}_findmycells_database.dbase'
        self._save_attr_to_disk(attr_id = 'database', filename = dbase_filename, child_attr_ids_to_del = ['project_configs'])
        configs_filename = f'{date}_findmycells_project.configs'
        self._save_attr_to_disk(attr_id = 'project_configs', filename = configs_filename, child_attr_ids_to_del = ['available_processing_modules'])
        
    
    def _save_attr_to_disk(self, attr_id: str, filename: str, child_attr_ids_to_del: List[str]) -> None:
        filepath = self.project_configs.root_dir.joinpath(filename)
        attribute_to_save = getattr(self, attr_id)
        for attr_id_to_del in child_attr_ids_to_del:
            delattr(attribute_to_save, attr_id_to_del)
        filehandler = open(filepath, 'wb')
        pickle.dump(attribute_to_save, filehandler)

        
    def _load_object_from_filepath(self, filepath: PosixPath) -> Union[Database, ProjectConfigs]:
        filehandler = open(filepath, 'rb')
        loaded_object = pickle.load(filehandler)
        return loaded_object

        
    def load_status(self,
                    project_configs_filepath: Optional[PosixPath]=None,
                    database_filepath: Optional[PosixPath]=None
                   ) -> None:
        if project_configs_filepath != None:
            assert type(project_configs_filepath) == PosixPath, '"project_configs_filepath" must be pathlib.Path object referring to a .configs file.'
            assert project_configs_filepath.suffix == '.configs', '"project_configs_filepath" must be pathlib.Path object referring to a .configs file.'
        else:
            project_configs_filepath = self._look_for_latest_status_file_in_dir(suffix = '.configs', dir_path = self.project_configs.root_dir)
        if database_filepath != None:
            assert type(database_filepath) == PosixPath, '"database_filepath" must be pathlib.Path object referring to a .dbase file'
            assert database_filepath.suffix == '.dbase', '"database_filepath" must be pathlib.Path object referring to a .dbase file'
        else:
            database_filepath = self._look_for_latest_status_file_in_dir(suffix = '.dbase', dir_path = self.project_configs.root_dir)
        if hasattr(self, 'project_configs'):
            delattr(self, 'project_configs')
        if hasattr(self, 'database'):
            delattr(self, 'database')
        self.project_configs = self._load_object_from_filepath(filepath = project_configs_filepath)
        self.project_configs.load_available_processing_modules()
        self.database = self._load_object_from_filepath(filepath = database_filepath)
        setattr(self.database, 'project_configs', self.project_configs)
        

    def split_file_ids_into_batches(self, file_ids: List[str], batch_size: int) -> List[List[str]]:
        if len(file_ids) % batch_size == 0:
            total_batches = int(len(file_ids) / batch_size)
        else:
            total_batches = int(len(file_ids) / batch_size) + 1
        file_ids_per_batch = []
        for batch in range(total_batches):
            if len(file_ids) >= batch_size:
                sampled_file_ids = random.sample(file_ids, batch_size)
            else:
                sampled_file_ids = file_ids.copy()
            file_ids_per_batch.append(sampled_file_ids)
            for elem in sampled_file_ids:
                file_ids.remove(elem)    
        return file_ids_per_batch


    def _look_for_latest_status_file_in_dir(self, suffix: str, dir_path: PosixPath) -> PosixPath:
        matching_filepaths = [filepath for filepath in dir_path.iterdir() if filepath.suffix == suffix]
        if len(matching_filepaths) == 0:
            raise FileNotFoundError(f'Could not find a "{suffix}" file in {dir_path}. Consider specifying the exact filepath!')
        else:
            date_strings = [filepath.name[:10] for filepath in matching_filepaths]
            dates = [datetime.strptime(date_str, '%Y_%m_%d') for date_str in date_strings]
            latest_date = max(dates)
            filepath_idx = dates.index(latest_date)
            latest_status_filepath = matching_filepaths[filepath_idx]
        return latest_status_filepath        
        
        
    def _assert_and_update_input(self, 
                                 processing_step_id: str,
                                 strategies: List[PreprocessingStrategy],
                                 strategy_configs: Optional[List[Dict]],
                                 processing_configs: Optional[Dict],
                                 file_ids: Optional[List[str]]
                                ) -> Tuple[List[Dict], Dict, List[str]]:
        self._assert_processing_step_input(processing_step_id = processing_step_id,
                                           strategies = strategies,
                                           strategy_configs = strategy_configs,
                                           processing_configs = processing_configs,
                                           file_ids = file_ids)
        strategy_configs = self._fill_strategy_configs_with_defaults_where_needed(strategies, strategy_configs)
        if processing_configs == None:
            if hasattr(self.project_configs, processing_step_id) == False:
                self.project_configs.add_processing_step_configs(processing_step_id = processing_step_id)
            processing_configs = getattr(self.project_configs, processing_step_id)
        processing_configs = self._fill_processing_configs_with_defaults_where_needed(processing_step_id, processing_configs)
        self.project_configs.add_processing_step_configs(processing_step_id, configs = processing_configs)
        file_ids = self.database.get_file_ids_to_process(input_file_ids = file_ids,
                                                         processing_step_id = processing_step_id,
                                                         overwrite = processing_configs['overwrite'])
        return strategy_configs, processing_configs, file_ids
            
        
    def _assert_processing_step_input(self, 
                                      processing_step_id: str,
                                      strategies: List[PreprocessingStrategy],
                                      strategy_configs: Optional[List[Dict]],
                                      processing_configs: Optional[Dict],
                                      file_ids: Optional[List[str]]
                                     ) -> None:
        assert type(strategies) == list, '"strategies" has to ba a list of ProcessingStrategy classes of the respective processing step!'
        if strategy_configs != None:
            assert type(strategy_configs) == list, '"strategy_configs" has to be None or a list of the same length as "strategies"!'
            assert len(strategy_configs) == len(strategies), '"strategy_configs" has to be None or a list of the same length as "strategies"!'
        else:
            strategy_configs = [None] * len(strategies)
        available_strategies = self.project_configs.available_processing_strategies[processing_step_id]
        for strat, config in zip(strategies, strategy_configs):
            assert strat in available_strategies, f'{strat} is not an available strategy for {processing_step_id}!'
            if config != None:
                strat().default_configs.assert_user_input(user_input = config)
        if processing_configs != None:
            processing_obj = self.project_configs.available_processing_objects[processing_step_id]()
            processing_obj.default_configs.assert_user_input(user_input = processing_configs)
        if file_ids != None:
            assert type(file_ids) == list, '"file_ids" has to be a list of strings referring to file_ids in the database!'
            for elem in file_ids:
                assert elem in self.database.file_infos['file_id'], f'{elem} is not a valid file_id!'
        
        
    def _fill_processing_configs_with_defaults_where_needed(self,
                                                            processing_step_id: str,
                                                            processing_configs: Dict
                                                           ) -> Dict:
        processing_obj = self.project_configs.available_processing_objects[processing_step_id]()
        return processing_obj.default_configs.fill_user_input_with_defaults_where_needed(user_input = processing_configs)                                              
             
        
    def _fill_strategy_configs_with_defaults_where_needed(self,
                                                          strategies: List[ProcessingStrategy],
                                                          strategy_configs: Optional[List[Dict]]
                                                         ) -> List[Dict]:
        all_final_configs = []
        if strategy_configs == None:
            for strat in strategies:
                default_configs = strat().default_configs.fill_user_input_with_defaults_where_needed(user_input = {})
                all_final_configs.append(default_configs)
        else:
            for strat, configs in zip(strategies, strategy_configs):
                full_configs = strat().default_configs.fill_user_input_with_defaults_where_needed(user_input = configs)
                all_final_configs.append(full_configs)
        return all_final_configs

In [ ]:
#| export

from pathlib import Path, PosixPath
from typing import List, Dict, Tuple, Optional, Union
from traitlets.traitlets import MetaHasTraits as WidgetType

from tqdm.notebook import tqdm
from datetime import datetime
import pickle
import ipywidgets as w
from IPython.display import display
from ipyfilechooser import FileChooser
import os
import pandas as pd


In [ ]:
class StrategyConfigurator:
    
    """
    This class implements the interface that let´s the user choose and 
    configurate the processing strategies. It will be placed inside of
    an accordion that is implemented in the `ProcessingStepPage`.
    It gets a list of all available processing strategies from the parent
    `ProcessingStepPage` and, thus, eventually from the `API` that checks
    for all available processing strategies in the corresponding processing
    submodule (e.g. "findmycells.preprocessing.strategies"). Upon initializing
    it´s dropdown widget, which let´s the user browser through the different
    available strategies, it also initializes an object of each strategy.
    This object can then be used to run it´s associated method 
    ".initialize_gui_configs_and_widget()" to build the specified widget, 
    using its `GUIConfigs` instance. Essentially, this contains a
    description of what the processing strategy does and, if applicable,
    widgets to specify all parameters that can be configurated for this 
    strategy. Finally, a "confirm & export" and a "remove" button allow
    the user to load or delete the current to or from the findmycells
    project, respectively.
    """
    
    def __init__(self,
                 available_strategy_classes: List,
                 parent_accordion: w.Accordion,
                 target_for_configs_export: List) -> None:
        self.available_strategy_classes = available_strategy_classes
        self.parent_accordion = parent_accordion
        self.target_for_configs_export = target_for_configs_export
        self.widget = self._initialize_widget()
        self._link_widgets_with_eventhandlers()

        
    def _initialize_widget(self) -> WidgetType:
        intro_label = w.HTML(value = ('Please select one of the available strategies in the dropdown menu below. '
                                       'Feel free to click through all listed strategies, as each of them will '
                                       'display a short description of the given strategy and also prompt you with '
                                       'some customization options (if there are any available for the respective '
                                       'strategy. Once you have made your choice & specified all customizable values, '
                                       'click on the "confirm selection & export configurations" to load it into your '
                                       'project. If you change your mind, you can simply click the "remove strategy" '
                                       'button to remove this strategy and all related configurations again. You can '
                                       'always add additional strategies if you minimize this accordion tab and open a '
                                       'new one. <br> Note: Strategies will be executed in the order that you add them to '
                                       'the accordion.'))
        spacer = w.Label(value = '', layout = {'height': '30px'})
        self.dropdown = self._initialize_dropdown()
        self.confirm_and_export_button = w.Button(description = 'confirm selection & export configurations', layout = {'width': '30%'})
        self.remove_button = w.Button(description = 'remove strategy', layout = {'width': '20%'}, disabled = True)
        self.displayed_strat_widget = w.VBox([self.dropdown.value.widget], layout = {'width': '95%'})
        widget = w.VBox([intro_label,
                         spacer,
                         w.HBox([self.dropdown, self.confirm_and_export_button, self.remove_button]),
                         spacer,
                         self.displayed_strat_widget])
        return widget
        
        
    def _link_widgets_with_eventhandlers(self) -> None:
        self.confirm_and_export_button.on_click(self._confirm_and_export_button_clicked)
        self.remove_button.on_click(self._remove_button_clicked)
        self.dropdown.observe(self._dropdown_option_changed, names = 'value')
        
    
    def _initialize_dropdown(self) -> WidgetType:
        dropdown_option_tuples = []
        for strategy_class in self.available_strategy_classes:
            strategy_obj = strategy_class()
            strategy_obj.initialize_gui_configs_and_widget()
            dropdown_option_tuples.append((strategy_obj.dropdown_option_value_for_gui, strategy_obj))
        return w.Dropdown(options = dropdown_option_tuples, layout = {'width': '50%'}) 
        
        
    def _get_own_position_idx_in_parent_accordion(self) -> int:
        return self.parent_accordion.children.index(self.widget)
        
        
    def _confirm_and_export_button_clicked(self, b) -> None:
        self._export_configs()
        self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = True)
        self._add_new_strategy_configurator_to_parent_accordion()
        self._change_own_accordion_tab_title(title = self.dropdown.value.dropdown_option_value_for_gui)
        
        
    def _add_new_strategy_configurator_to_parent_accordion(self) -> None:
        new_strategy_configurator = StrategyConfigurator(available_strategy_classes = self.available_strategy_classes,
                                                         parent_accordion = self.parent_accordion,
                                                         target_for_configs_export = self.target_for_configs_export)
        self.parent_accordion.children = self.parent_accordion.children + (new_strategy_configurator.widget, )
        position_idx = len(self.parent_accordion.children) - 1
        self.parent_accordion.set_title(position_idx, 'Expand me to add a processing strategy')
                                                         
                                                         
    def _change_own_accordion_tab_title(self, title: str) -> None:
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.parent_accordion.set_title(position_idx, title)

        
    def _export_configs(self) -> None:
        selected_strategy_obj = self.dropdown.value
        current_configs = selected_strategy_obj.gui_configs.export_current_config_values()
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.target_for_configs_export.insert(position_idx, (selected_strategy_obj.__class__, current_configs))      
        
        
    def _change_disable_settings_of_customizable_widgets(self, disable_customizable_widgets: bool) -> None:
        self.remove_button.disabled = not disable_customizable_widgets
        self.dropdown.disabled = disable_customizable_widgets
        self.confirm_and_export_button.disabled = disable_customizable_widgets
        for widget in self.displayed_strat_widget.children[0].children:
            if hasattr(widget, 'disabled'):
                widget.disabled = disable_customizable_widgets
            elif type(widget) == w.HBox:
                if hasattr(widget.children[0], 'disabled'):
                    widget.children[0].disabled = disable_customizable_widgets
        
        
    def _remove_button_clicked(self, b) -> None:
        self._remove_configs()
        if len(self.parent_accordion.children) == 1:
            self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = False)
            self._change_own_accordion_tab_title(title = 'Expand me to add a processing strategy')
        else:
            currently_present_accordion_tabs = len(self.parent_accordion.children)
            currently_confirmed_strategies = len(self.target_for_configs_export)
            tabs_available_for_selection = currently_present_accordion_tabs - currently_confirmed_strategies
            if tabs_available_for_selection > 1:
                self._remove_own_tab_from_parent_accordion()
            else:
                self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = False)
                self._change_own_accordion_tab_title(title = 'Expand me to add a processing strategy')
                
                
    def _remove_own_tab_from_parent_accordion(self) -> None:
        tmp_children = list(self.parent_accordion.children)
        tmp_titles = []
        for idx in range(len(tmp_children)):
            tmp_titles.append(self.parent_accordion.get_title(idx))
        position_idx = self._get_own_position_idx_in_parent_accordion()
        tmp_children.pop(position_idx)
        tmp_titles.pop(position_idx)
        self.parent_accordion.children = tuple(tmp_children)
        for idx, title in enumerate(tmp_titles):
            self.parent_accordion.set_title(idx, title)
            
            
    def _remove_configs(self) -> None:
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.target_for_configs_export.pop(position_idx)
                
                
    def _dropdown_option_changed(self, change) -> None:
        new_selection = change.new
        self.displayed_strat_widget.children = (new_selection.widget, )

In [ ]:
#| export

class PageButtonBundle(ABC):
    
    
    @abstractmethod
    def _initialize_page_content(self) -> WidgetType:
        pass
    
    
    def __init__(self, bundle_id: str, page_screen: WidgetType, all_navigator_buttons: List, api: API) -> None:
        self.bundle_id = bundle_id
        self.gui_page_screen = page_screen
        self.all_navigator_buttons = all_navigator_buttons
        self.api = api
        self.navigator_button = self._initialize_navigator_button()
        self.page_content = self._initialize_page_content()
        
        
    def _initialize_navigator_button(self) -> WidgetType:
        navigator_button = w.Button(description = self.bundle_id, style = {'button_color': 'gray'})
        navigator_button.on_click(self._navigator_button_clicked)
        return navigator_button
    
    
    def _navigator_button_clicked(self, b) -> None:
        for button in self.all_navigator_buttons:
            button.style.button_color = 'gray'
        self.navigator_button.style.button_color = 'skyblue'
        self.gui_page_screen.children = (self.page_content, )

In [ ]:
#| export

class OverviewPage(PageButtonBundle):
        
    def _initialize_page_content(self) -> WidgetType:
        overview_intro_text = w.HTML(value = ('This is the overview page for your findmycells '
                                              'project. You will find everything relevant regarding '
                                              'the organization of your project in the tabs below. '
                                              'If you have just started your findmycells project, '
                                              'please make sure to follow the instructions in the '
                                              '"project files" tab before you can get started.'))
        project_files_tab_widget = self._initialize_project_files_tab_widget()
        save_load_project_tab_widget = self._initialize_save_load_project_tab_widget()
        browse_file_histories_tab_widget = self._initialize_browse_file_histories_tab_widget()
        self._bind_buttons_to_functions()
        tabs = w.Tab([save_load_project_tab_widget, 
                      project_files_tab_widget, 
                      browse_file_histories_tab_widget], selected_index = 1)
        tabs.set_title(0, 'save & load project')
        tabs.set_title(1, 'project files')
        tabs.set_title(2, 'browser file histories')
        page_content_widget = w.VBox([overview_intro_text, tabs])
        return page_content_widget
        
        
    def _initialize_project_files_tab_widget(self) -> WidgetType:
        intro_text = w.HTML(value = ('Just started a new project? Great! Before you can get started with '
                                     'the processing of your data, you need to associate the corresponding '
                                     'files with your findmycells project. Unfortunately, this requires you '
                                     'to arrange your files in a very rigid structure of directories (this '
                                     'will be fixed in a later version). But don´t worry, if you click on '
                                     'the "expand me if you need a detailed guide to prepare your data!" '
                                     'widget below you will find a comprehensive overview on how this tree '
                                     'of directories has to look like. Once you have your data arranged '
                                     'accordingly, just click the "update project files" button below. You '
                                     'can also always come back to this page & hit the button again in order '
                                     'to update the files associated with your project. Findmycells will then '
                                     'automatically identify files that have been deleted or added to the '
                                     'directories and remove them from or add them to your current project, '
                                     'respectively. To get an overview of which files are currently associ'
                                     'ated with your project, just click the "display current project files" '
                                     'button right next to the "update project files" button. If you are inter'
                                     'ested in more detailed information about each file, for instance its '
                                     'processing history, please head over to the "file histories" tab.'))
        guide_accordion = self._initialize_guide_accordion_widget()
        self.update_project_files_button = w.Button(description = 'update project files', icon = 'refresh')
        self.display_current_project_files_button = w.Button(description = 'display current project files')
        buttons = w.HBox([self.update_project_files_button, self.display_current_project_files_button])
        self.current_project_files_output = w.Output()
        project_files_tab_widget = w.VBox([intro_text,
                                           guide_accordion,
                                           buttons,
                                           self.current_project_files_output])
        return project_files_tab_widget
    
    
    def _initialize_guide_accordion_widget(self) -> WidgetType:
        # ToDo: write detailed description of directory tree
        # ToDo: add sample image of how this could look like
        # ToDo: what to do for walkthrough through data
        # ToDo: hint that just clicking the button with an empty microscopy dir launches sample tree
        general_info = w.HTML(value = 'space for some general infos')
        detailed_infos = w.HTML(value = 'all details come here')
        sample_image = w.HTML(value = 'this shall be replaced by the sample image')
        whole_guide = w.VBox([general_info, 
                              detailed_infos,
                              sample_image])
        guide_accordion = w.Accordion([whole_guide], selected_index = None)
        guide_accordion.set_title(0, 'expand me if you need a detailed guide to prepare your data!')
        return guide_accordion
        
        
    def _initialize_save_load_project_tab_widget(self) -> WidgetType:
        intro_text = w.HTML(value = ('Here you can either save the progress of your currently '
                                     'running project or load a previously saved project.'))
        save_project_widget = self._initialize_save_project_widget()
        load_project_widget = self._initialize_load_project_widget()
        accordion = w.Accordion([save_project_widget, load_project_widget], selected_index = 0)
        accordion.set_title(0, 'save')
        accordion.set_title(1, 'load')
        save_load_project_tab_widget = w.VBox([intro_text, accordion])
        return save_load_project_tab_widget
    
    
    def _initialize_save_project_widget(self) -> WidgetType:
        save_description = w.HTML(value = ('Clicking the following "save" button will save '
                                           'your current project, including all configurations '
                                           'and processing progress. The file will automatically '
                                           'be written as a ".configs" file to the root directory '
                                           'you specified, with the current date as prefix.'))
        self.save_project_button = w.Button(description = 'save project', icon = 'save')
        save_project_widget = w.VBox([save_description, self.save_project_button])
        return save_project_widget
    
    
    def _initialize_load_project_widget(self) -> WidgetType:
        load_description = w.HTML(value = ('You already have a findmycells project to load? Great! '
                                           'Please make sure to choose the corresponding root dir'
                                           'rectory in which you previously created and run your '
                                           'project upon starting this GUI. If you have specified a '
                                           'different root directory, simply restart the GUI. When '
                                           'you are in the correct root directory, simply click the '
                                           '"load project" button to load your project and the last '
                                           'status you have saved.'))
        self.load_project_button = w.Button(description = 'load project', icon = 'upload')
        # ToDo: 
        #   For the moment you should only load a project from its own root dir.
        #   However, it should be possible in later versions to provide the filepath(s)
        #   to the file(s) that fmc created upon saving the project. This would, consequently,
        #   require the use of filechooser(s). I think it was possible to restrict the selection
        #   to only files with a specific extension (to make sure the user selects the correct
        #   file(s)). In addition, it might be possible to re-configure the default filepath
        #   of the current root dir. 
        load_project_widget = w.VBox([load_description,
                                      self.load_project_button])
        return load_project_widget
    
        
    def _initialize_browse_file_histories_tab_widget(self) -> WidgetType:
        intro_text = w.HTML(value = ('Findmycells keeps a detailed track of how and when your '
                                     'files are processed. Using the widget below, you are '
                                     'able to browser through this history for all files in your '
                                     'project. Note: Unfortunately, this interactive widget is '
                                     'not yet implemented in your current version. Nevertheless, '
                                     'findmycells already tracks the processing history of your '
                                     'files and if you have some experience with python, you can '
                                     'access them at: self.api.database.file_histories, where '
                                     '"self" refers to the GUI object you have instantiated.'))
        self.file_histories_id_dropdown = w.Dropdown(description = 'History of file ID:', 
                                                     style = {'description_width': 'initial'},
                                                     layout = {'width': '66%'})
        self.display_file_history_button = w.Button(description = 'show history', layout = {'width': '33%'})
        dropdown_button_hbox = w.HBox([self.file_histories_id_dropdown, self.display_file_history_button])
        self.file_history_output = w.Output()
        browse_file_hostories_tab_widget = w.VBox([intro_text,
                                                   dropdown_button_hbox,
                                                   self.file_history_output])
        return browse_file_hostories_tab_widget
                                      
        
    def _bind_buttons_to_functions(self) -> None:
        self.update_project_files_button.on_click(self._update_project_files_button_clicked)
        self.display_current_project_files_button.on_click(self._display_current_project_files_button_clicked)
        self.save_project_button.on_click(self._save_project_button_clicked)
        self.load_project_button.on_click(self._load_project_button_clicked)
        self.display_file_history_button.on_click(self._display_file_history_button_clicked)
        
        
    def _update_project_files_button_clicked(self, b) -> None:
        self.api.update_database_with_current_source_files()
        self.api.database.compute_file_infos()
        self._update_options_for_file_histories_id_dropdown()
        self._display_current_project_files_button_clicked('simulate a click')
        
    
    def _update_options_for_file_histories_id_dropdown(self) -> None:
        self.file_histories_id_dropdown.options = self.api.database.file_infos['file_id']
        
        
    def _display_current_project_files_button_clicked(self, b) -> None:
        file_infos_df = pd.DataFrame(data = self.api.database.file_infos)
        refreshed_datetime = datetime.now()
        with self.current_project_files_output:
            self.current_project_files_output.clear_output()
            print(f'Data refreshed at {refreshed_datetime:%H:%M:%S} on {refreshed_datetime:%d.%m.%Y}')
            print('\n\n')
            display(file_infos_df)

            
    def _save_project_button_clicked(self, b) -> None:
        self.api.save_status()
    
    
    def _load_project_button_clicked(self, b) -> None:
        self.api.load_status()
        
    
    def _display_file_history_button_clicked(self, b) -> None:
        # ToDo: 
        #   Needs to be implemented. File histories are implemented, yet
        #   it needs to be checked how to best visualize them, as they 
        #   consist of two parts, a pd.DataFrame that tracks the general
        #   processing information (what strategy was run when & was a 
        #   given processing step completed), and a dictionary that lists
        #   the exact parameters (user settings and determined by fmc) 
        #   for each processing step individually
        with self.file_history_output:
            self.file_history_output.clear_output()
            print('Not implemented yet!')

In [ ]:
#| export

class ProcessingStepPage(PageButtonBundle):
    
    #ToDo: 
    # For preprocessing, widgets to configure the DataReaders (microscopy_configs & roi_configs) need to be added!
        
    def _initialize_page_content(self) -> WidgetType:
        self.exported_strategies_with_configs = []
        self._initialize_strategy_selection_accordion()
        self._initialize_processing_configs_widget()
        self._initialize_trigger_widget_elements()
        self._bind_buttons_to_functions()
        widget = w.VBox([self.strat_selection_accordion,
                         self.processing_configs_widget,
                         w.HBox([self.file_ids_range, self.run])])
        return widget
    
    
    def _initialize_processing_configs_widget(self) -> None:
        processing_object_class = self.api.project_configs.available_processing_objects[self.bundle_id] 
        self.processing_obj = processing_object_class()
        self.processing_obj.initialize_gui_configs_and_widget()
        self.confirm_and_processing_configs = w.Button(description = 'confirm & export configurations',
                                                       layout = {'width': '30%'},
                                                       style = {'description_width': 'initial'})
        self.refine_processing_configs = w.Button(description = 'refine configurations',
                                                  disabled = True,
                                                  layout = {'width': '30%'},
                                                  style = {'description_width': 'initial'})
        self.processing_configs_widget = w.VBox([self.processing_obj.widget,
                                                 w.HBox([self.confirm_and_processing_configs, self.refine_processing_configs])])
        
        
    def _initialize_strategy_selection_accordion(self) -> None:
        available_strategy_classes = self.api.project_configs.available_processing_strategies[self.bundle_id]        
        self.strat_selection_accordion = w.Accordion()
        initial_strat_configurator = StrategyConfigurator(available_strategy_classes=available_strategy_classes,
                                                          parent_accordion=self.strat_selection_accordion,
                                                          target_for_configs_export=self.exported_strategies_with_configs)
        self.strat_selection_accordion.children = self.strat_selection_accordion.children + (initial_strat_configurator.widget, )
        self.strat_selection_accordion.set_title(0, 'Expand me to add a processing strategy')
        
        
    def _initialize_trigger_widget_elements(self) -> None:
        all_file_ids = self.api.database.file_infos['file_id']
        self.file_ids_range = w.SelectionRangeSlider(description = 'Select range of file IDs to process: ',
                                                     options = all_file_ids,
                                                     value = (all_file_ids[0], all_file_ids[-1]),
                                                     layout = {'width': '75%'},
                                                     style = {'description_width': 'initial'})
        self.run = w.Button(description = f'Launch {self.bundle_id}',
                            icon = 'rocket',
                            disabled = True,
                            layout = {'width': '25%'},
                            style = {'description_width': 'initial', 'button_color': 'orange'}
                           )

    
    def _bind_buttons_to_functions(self) -> None:
        self.confirm_and_processing_configs.on_click(self._confirm_and_processing_configs_clicked)
        self.refine_processing_configs.on_click(self._refine_processing_configs_clicked)
        self.run.on_click(self._run_clicked)
        
        
    def _confirm_and_processing_configs_clicked(self, b):
        self.processing_configs = self.processing_obj.gui_configs.export_current_config_values()
        self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = True)
        
    
    def _change_disable_settings_of_customizable_widgets(self, disable_customizable_widgets: bool) -> None:
        self.run.disabled = not disable_customizable_widgets
        self.refine_processing_configs.disabled = not disable_customizable_widgets
        self.confirm_and_processing_configs.disabled = disable_customizable_widgets
        for widget in self.processing_obj.widget.children:
            if hasattr(widget, 'disabled'):
                widget.disabled = disable_customizable_widgets
            elif type(widget) == w.HBox:
                if hasattr(widget.children[0], 'disabled'):
                    widget.children[0].disabled = disable_customizable_widgets
                    
                    
    def _refine_processing_configs_clicked(self, b) -> None:
        self.processing_configs = {}
        self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = False)
        
        
    def _run_clicked(self, b) -> None:
        file_ids = self._get_file_ids()
        strategies, strategy_configs = self._get_strategies_and_configs()
        self.run.disabled = True
        self.refine_processing_configs.disabled = True
        self.api.preprocess(strategies = strategies,
                            strategy_configs = strategy_configs,
                            processing_configs = self.processing_configs,
                            microscopy_reader_configs = None,
                            roi_reader_configs = None,
                            file_ids = file_ids)
        self.run.disabled = False
        self.refine_processing_configs.disabled = False
        
        
    def _get_file_ids(self) -> List[str]:
        all_file_ids = self.api.database.file_infos['file_id']
        start_idx = all_file_ids.index(self.file_ids_range.value[0])
        end_idx = all_file_ids.index(self.file_ids_range.value[1])
        file_ids = all_file_ids[start_idx : end_idx+1]
        return file_ids
    
    
    def _get_strategies_and_configs(self) -> Tuple[List[ProcessingStrategy], List[Dict]]:
        strategies, strategy_configs = zip(*self.exported_strategies_with_configs)
        strategies = list(strategies)
        strategy_configs = list(strategy_configs)
        return strategies, strategy_configs

In [ ]:
#| export

class GUI:
    
    def __init__(self) -> None:
        self.displayed_widget = self._initialize_start_screen()
        
    
    def _initialize_start_screen(self) -> WidgetType:
        welcome_label_line_1 = w.Label(value = 'Welcome to findmycells, glad you´re here! :-)')
        welcome_label_line_2 = w.Label(value = 'Please start by selecting the root directory for your project below.')
        welcome_label_line_3 = w.Label(value = 'When you´re happy with your selection, click "confirm" and we are ready to go!')
        current_working_dir = os.getcwd()
        #self.root_dir = FileChooser(current_working_dir, show_only_dirs = True)
        self.root_dir = FileChooser('/mnt/c/Users/dsege/Downloads/fmc_test_project/', show_only_dirs = True)
        confirm_root_dir_selection = w.Button(description = 'confirm')
        confirm_root_dir_selection.on_click(self._confirm_root_dir_selection)
        return w.VBox([welcome_label_line_1, welcome_label_line_2, welcome_label_line_3, self.root_dir, confirm_root_dir_selection])
   
    
    def _confirm_root_dir_selection(self, b) -> None:
        selected_root_dir_path = Path(self.root_dir.value)
        assert selected_root_dir_path.is_dir()
        self.api = API(project_root_dir = selected_root_dir_path)
        self._initialize_main_screen()
        
        
    def _initialize_main_screen(self) -> None:
        self.page_screen = w.VBox()
        self.api.update_database_with_current_source_files()
        self._initialize_page_bundles()
        navigator_bar = w.HBox(self.navigator_buttons)
        self.main_screen = w.VBox([navigator_bar, self.page_screen])
        self._refresh_displayed_widget(new_widget = self.main_screen)
        
        
        
    def _initialize_page_bundles(self) -> None:
        self.navigator_buttons = []
        self.overview_page = OverviewPage(bundle_id = 'overview',
                                          page_screen = self.page_screen,
                                          all_navigator_buttons = self.navigator_buttons,
                                          api = self.api)
        self.navigator_buttons.append(self.overview_page.navigator_button)
        for available_processing_module_name in self.api.project_configs.available_processing_modules.keys():
            processing_step_page = ProcessingStepPage(bundle_id = available_processing_module_name,
                                                      page_screen = self.page_screen,
                                                      all_navigator_buttons = self.navigator_buttons,
                                                      api = self.api)
            self.navigator_buttons.append(processing_step_page.navigator_button)
            attr_id = f'{available_processing_module_name}_page'
            setattr(self, attr_id, processing_step_page)

        
    def _refresh_displayed_widget(self, new_widget: WidgetType) -> None:
        self.displayed_widget.children = (new_widget, )

In [ ]:
#| hide

gui = GUI()
gui.displayed_widget

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

SyntaxError: invalid syntax (<unknown>, line 173)